In [ ]:
import os

from tqdm.auto import tqdm
import numpy as np
import scqubits as scq
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

In [ ]:
qub_name = "Design1"

os.makedirs(f"../../result/{qub_name}/image", exist_ok=True)
os.makedirs(f"../../result/{qub_name}/web", exist_ok=True)

# Scan params

In [ ]:
EJb = (2.0, 10.0)
EC = 0.8
ELb = (0.35, 2.0)
avoid_freqs = [7.2, 5.9]
# avoid_freqs = []

In [ ]:
Temp = 60e-3
Q_cap = 1.0e5
Q_ind = 1.0e7
x_qp = 1.0e-8

t1s = []
m01s = []
paramss = []
collisions = []

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(EJb[0], EC, ELb[0], flux=0.5, cutoff=40, truncated_dim=5)
for EJ in tqdm(np.arange(EJb[0], EJb[1], 0.05), desc="EJ"):
    for EL in np.arange(ELb[0], ELb[1], 0.05):
        fluxonium.EJ = EJ
        fluxonium.EC = EC
        fluxonium.EL = EL

        spectData = fluxonium.eigensys(evals_count=5, return_spectrumdata=True)
        eval_eng, eval_sys = spectData.energy_table, spectData.state_table
        e0x = eval_eng - eval_eng[0]
        e1x = eval_eng - eval_eng[1]

        freqs = np.array(avoid_freqs)[None, :]
        all_collision = np.all(
            np.logical_or(
                np.any(np.abs(e0x[:, None] - freqs) < 0.3, axis=0),
                np.any(np.abs(e1x[:, None] - freqs) < 0.3, axis=0),
            )
        )

        e01 = eval_eng[1] - eval_eng[0]
        for i in range(eval_eng.shape[0]):
            for j in range(i + 1, eval_eng.shape[0]):
                eij = eval_eng[j] - eval_eng[i]
                if (i != 0 or j != 1) and np.abs(eij - e01) < 0.3:
                    all_collision = True
                    break
            if all_collision:
                break

        elements = fluxonium.n_operator(energy_esys=(eval_eng, eval_sys))
        t1_eff = fluxonium.t1_effective(
            esys=(eval_eng, eval_sys),
            noise_channels=[
                ("t1_capacitive", dict(Q_cap=Q_cap)),
                ("t1_inductive", dict(Q_ind=Q_ind)),
                ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
            ],
            common_noise_options=dict(i=1, j=0, T=Temp),
        )

        t1s.append(t1_eff)
        m01s.append(elements[0, 1])
        paramss.append((EJ, EC, EL))
        collisions.append(all_collision)
t1s = np.array(t1s)
m01s = np.array(m01s)
paramss = np.array(paramss)
collisions = np.array(collisions)

In [ ]:
# 將數據轉換為DataFrame
data = pd.DataFrame(
    {
        "EJ": paramss[:, 0],
        "EC": paramss[:, 1],
        "EL": paramss[:, 2],
        "t1": t1s,
        "m01": np.abs(m01s),
        "collision": collisions,
    }
)

# 添加標籤
data["Label"] = data.apply(
    lambda row: f"EJ={row['EJ']:.2f}, EC={row['EC']:.3f}, EL={row['EL']:.3f}", axis=1
)

# 繪製散點圖
fig = px.scatter(
    data,
    x="m01",
    y="t1",
    color="collision",
    color_discrete_map={True: "red", False: "blue"},
    log_x=True,
    log_y=True,
    hover_name="Label",
    title=f"T1 vs Matrix Element 0-1 (Q_cap={Q_cap:.1e}, Q_ind={Q_ind:.1e}, x_qp={x_qp:.1e})",
    labels={"Matrix Element 0-1": "m01", "T1 (us)": "t1"},
)
fig.update_traces(marker=dict(size=5))

max_id = np.argmax([0.0 if c else t1 for t1, c in zip(t1s, collisions)])

# Add annotation for the max_id point
max_point = data.iloc[max_id]
fig.add_annotation(
    x=np.log10(max_point["m01"]),
    y=np.log10(max_point["t1"]),
    text=max_point["Label"],
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    xref="x",
    yref="y",
)

fig.update_layout(
    xaxis_title="Matrix Element 0-1",
    yaxis_title="T1 (us)",
    xaxis=dict(tickformat=".0e", type="log"),  # Ensure axis type is log
    yaxis=dict(tickformat=".0e", type="log"),  # Ensure axis type is log
    title_x=0.501,
    template="plotly_white",
    showlegend=True,
    width=1100,
    height=750,
)
pass
# fig.show()

In [ ]:
record_chips = {
    "S001": {
        "params": (4.6770, 0.8058, 1.1761),
        "t1": 1.1,
    },
    "2DQ9/Q5": {
        "params": (8.47885, 0.909689, 1.0834447),
        "t1": 14.25,
    },
}

In [ ]:
for chip, data in record_chips.items():
    fluxonium.EJ = data["params"][0]
    fluxonium.EC = data["params"][1]
    fluxonium.EL = data["params"][2]
    fluxonium.flux = 0.5
    element = fluxonium.n_operator(energy_esys=True)

    m01 = np.abs(element[0, 1])

    fig.add_annotation(
        x=np.log10(m01),
        y=np.log10(data["t1"]),
        text=chip,
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-40,
        xref="x",
        yref="y",
    )
fig.show()

In [ ]:
fig.write_html(
    f"../../result/{qub_name}/web/t1vsM01_EC{EC:.2f}.html", include_plotlyjs="cdn"
)
fig.write_image(f"../../result/{qub_name}/image/t1vsM01_EC{EC:.2f}.png", format="png")

In [ ]:
EJ, EC, EL = paramss[max_id]
# EJ, EC, EL = 4.4, 1.4, 0.35
EJ, EC, EL

In [ ]:
params = (EJ, EC, EL)

flxs = np.linspace(0.0, 1.0, 300)

In [ ]:
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=40, truncated_dim=10)
spectrumData = fluxonium.get_matelements_vs_paramvals(
    operator="n_operator", param_name="flux", param_vals=flxs, evals_count=10
)

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(10) if j > i]

eval_eng = spectrumData.energy_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=eval_eng[:, j] - eval_eng[:, i],
            mode="lines",
            name=f"{i} - {j}",
            line=dict(width=2),
        )
    )
for freq in avoid_freqs:
    fig.add_hline(y=freq, line_color="black", line_width=2, line_dash="dash")
fig.update_yaxes(range=[0.0, 8.0])

fig.show()

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(3) if j > i]

matrixelements = spectrumData.matrixelem_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=np.abs(matrixelements[:, i, j]),
            mode="lines",
            name=f"{i}-{j}",
            line=dict(width=2),
        )
    )
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Matrix elements",
)

fig.show()

In [ ]:
# Temp = 60e-3
# Q_cap = 1.0e5
# Q_ind = 1.0e7
# x_qp = 1.0e-8

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=30, truncated_dim=5)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name="flux",
    param_vals=flxs,
    xlim=([flxs.min(), flxs.max()]),
    common_noise_options=dict(i=1, j=0, T=Temp),
    noise_channels=[
        ("t1_capacitive", dict(Q_cap=Q_cap)),
        ("t1_inductive", dict(Q_ind=Q_ind)),
        ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
    ],
)
_ = T1_eff_ax.set_ylim(None, 1e6)